In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={    
}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    inputs =    
    [ 
      { STEP_NAME: "data_load", ENTITY_NAME: "meta_cifar10_datasets"}, # meta information of dataset from data_load step
      { STEP_NAME: "data_prep", ENTITY_NAME: "cifar10_test_dataset"}, # dataset for detector testing from data_prep step    
      { STEP_NAME: "model_pack", ENTITY_NAME: "bento_service"}  # stored BentoService from model_pack
    ],
    tmp_outputs =
    [
        { ENTITY_NAME: "meta_cifar10_datasets"}, # extracted temporary meta information of dataset from data_load step
        { ENTITY_NAME: "cifar10_test_dataset" },  # ground-true test dataset files for use in next substep
        { ENTITY_NAME: "cifar10_inference_result_dataset" } # predicted test dataset files for use in next substep
    ],
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# specify all notebook wide libraries imports here
# Sinara lib imports is left in the place of their usage
import json
from tqdm import tqdm
import os
import os.path as osp
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
# run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

### Loading REST bento_service 

#### Get and load bentoservice

In [ ]:
from sinara.bentoml import load_bentoservice

# read trained model
print('read trained model')
inputs_model_pack = substep.inputs(step_name = "model_pack")
bento_service = load_bentoservice(inputs_model_pack.bento_service)

### Loading test datasets (from step data_prep)

In [ ]:
from sinara.archive import SinaraArchive
import json

# copy test data from previos step (data_prep) to tmp
inputs_data_prep = substep.inputs(step_name = "data_prep")
inputs_data_load = substep.inputs(step_name = "data_load")
tmp_outputs = substep.tmp_outputs()

archive = SinaraArchive(spark)
archive.unpack_files_from_store_to_tmp(store_path=inputs_data_prep.cifar10_test_dataset, tmp_dir=tmp_outputs.cifar10_test_dataset)
archive.unpack_files_from_store_to_tmp(store_path=inputs_data_load.meta_cifar10_datasets, tmp_dir=tmp_outputs.meta_cifar10_datasets)

## Inference by REST API bentoservice

### Get meta information cifar10 dataset

In [ ]:
with open(osp.join(tmp_outputs.meta_cifar10_datasets, 'meta_cifar10_datasets.json'), 'r') as f:
   categories = json.load(f)

### Get image pathes for cifar10 test dataset

In [ ]:
dir_test_cifar10_dataset = tmp_outputs.cifar10_test_dataset
class_ids = os.listdir(dir_test_cifar10_dataset)
test_cifar10_dataset = []
for class_id in class_ids:
    # Get images from train dataset
    for image_name in os.listdir(osp.join(dir_test_cifar10_dataset, class_id)):        
        image_path = osp.join(dir_test_cifar10_dataset, class_id, image_name)
        if osp.isdir(image_path):
            continue
        test_cifar10_dataset.append(image_path)

### Inference classificator on a test dataset for val step

In [ ]:
out_predict_cifar10_test_dataset = []

for image_path in tqdm(test_cifar10_dataset, desc='process predict'):
    image_name = osp.basename(image_path)
    ground_true_class_index = int(osp.basename(osp.dirname(image_path)))
    ground_true_class_name = categories[str(ground_true_class_index)]
    predict_result = bento_service.predict(image_path)
    predict_class_index = predict_result["class_ids"][0]
    predict_class_name = predict_result["class_names"][0]
    predict_class_score = predict_result["class_scores"][0]
    out_predict_cifar10_test_dataset.append(
        {"image_path": image_path,
         "image_name": image_name,
         "ground_true_class_index": ground_true_class_index,
         "ground_true_class_name": ground_true_class_name,
         "predict_class_index": predict_class_index,
         "predict_class_name": predict_class_name,
         "predict_class_score": predict_class_score}
    )
    
# save result of predict for cifar10 test datasets
with open(osp.join(tmp_outputs.cifar10_inference_result_dataset, 'val_cifar10_dataset.json'), 'w') as f:
    json.dump(out_predict_cifar10_test_dataset, f)

### Preview of classification on an image

In [ ]:
# Let's view more images in a grid format
# Define the dimensions of the plot grid 
W_grid = 5
H_grid = 5

# fig, axes = plt.subplots(H_grid, W_grid)
# subplot return the figure object and axes object
# we can use the axes object to plot specific figures at various locations
fig, axes = plt.subplots(H_grid, W_grid, figsize = (10,10))

axes = axes.ravel() # flaten the 15 x 15 matrix into 225 array

n_images = len(out_predict_cifar10_test_dataset) # get the length of the  dataset

# Select a random number from 0 to n_train
for i in range(W_grid * H_grid): # create evenly spaces variables 
    # Select a random number
    index = np.random.randint(0, n_images)
    # read and display an image with the selected index
    predict_sample = out_predict_cifar10_test_dataset[index]
    img_path = predict_sample["image_path"]
    ground_true_class_name = predict_sample["ground_true_class_name"]
    predict_class_name = predict_sample["predict_class_name"]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[i].imshow(img)
    axes[i].set_title(f"GT: {ground_true_class_name} \n Predict:{predict_class_name}", fontsize = 8)
    axes[i].axis('off')

plt.subplots_adjust(hspace=0.4)

In [ ]:
# stop spark
SinaraSpark.stop_session()